<a href="https://colab.research.google.com/github/seyedalicheraghi/Image-Classification/blob/master/Inference_TRT_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Next step after producing the TRT Engine is to run it on an image to check its result. This session provide the code to achieve this goal.

### Install requried libraries in Google Colab
---

In [1]:
!pip install pycuda # install cuda
!pip install tensorrt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 17.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 9.6 MB/s 
     |████████████████████████████████| 71 kB 10.8 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=646530 sha256=2730de5c816978d84be0eaabd7ae07de0a4d397812a97fa492362b1dd417bc0c
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.13-py2.py3-none-any.whl size=66024 sha256=3dfe6b4e18897a60891dee44b5e606500b44b30b07dbe12502e1de8088f1dfaf
  Stored in directory: /root/.cache/pip/wheels/b5/c1/bb/26ba70fb9d10f195249ef4e170a92ae83e7534e55b67786fd9
Successfully built pycuda pytools
Looking in indexes: https://pypi

#### Check TensorRT version
---

In [6]:
import tensorrt as trt
trt.__version__

'8.5.2.2'

#### Import requried libraries
---

In [2]:
from __future__ import print_function
import os
import argparse
import tensorrt as trt
import random
import argparse
import os

import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
import tensorrt as trt
import numpy as np
from PIL import Image

#### Define constants

---



In [13]:
engine_path = 'model.trt'
input_name = 'conv2d_input'
Input_shape = (3,256,256)

**Step 1:** Load the engine we created in previous steps.

In [3]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)
print("Load the created tensorrt engine")
with open(engine_path, 'rb') as f:
    engine_data = f.read()
engine = trt_runtime.deserialize_cuda_engine(engine_data)

Load the created tensorrt engine


In [8]:
# Simple helper data class that's a little nicer to use than a 2-tuple.
class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()

In [9]:
# Allocates all buffers required for an engine, i.e. host/device inputs/outputs.
def allocate_buffers(engine):
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        # Allocate host and device buffers
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        # Append the device buffer to device bindings.
        bindings.append(int(device_mem))
        # Append to the appropriate list.
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
    return inputs, outputs, bindings, stream

In [11]:
inputs, outputs, bindings, stream = allocate_buffers(engine)
context = engine.create_execution_context()

<ipython-input-9-4e2a1eedc286>:8: DeprecationWarning: Use get_tensor_shape instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
<ipython-input-9-4e2a1eedc286>:8: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
<ipython-input-9-4e2a1eedc286>:9: DeprecationWarning: Use get_tensor_dtype instead.
  dtype = trt.nptype(engine.get_binding_dtype(binding))
<ipython-input-9-4e2a1eedc286>:16: DeprecationWarning: Use get_tensor_mode instead.
  if engine.binding_is_input(binding):


In [14]:
test_image = np.float32(np.random.rand(Input_shape[0] ,Input_shape[1], Input_shape[2]))

inputs[0].host = test_image

In [21]:
def do_inference_v2(context, bindings, inputs, outputs, stream):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # # Run inference.
    # context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # # Transfer predictions back from the GPU.
    # [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # # Synchronize the stream
    # stream.synchronize()
    # # Return only the host outputs.
    # return [out.host for out in outputs]

In [22]:
trt_outputs = do_inference_v2(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)

LogicError: ignored